In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip3 install python-mnist
!wget https://github.com/sorki/python-mnist/blob/master/bin/mnist_get_data.sh
!mnist_get_data.sh

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys

import torch
import matplotlib.pylab as plt
from tqdm.notebook import tqdm, trange
import numpy as np
from scipy.spatial.distance import cdist
from mnist import MNIST
from IPython.display import clear_output

# sys.path.append('/content/drive/MyDrive/Documents/OPTAMI/OPTAMI')
# sys.path.append('/content/drive/MyDrive/Documents/OPTAMI')
from OPTAMI import PrimalDualAccelerated
from run_pd_experiment import *

In [3]:
n = 784
m = int(np.sqrt(n))

In [4]:
device = 'cpu'

In [5]:
A_matrix = calculate_A_matrix(n)
A_A_T = A_matrix @ A_matrix.T

Building matrix A: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 784/784 [00:02<00:00, 343.93it/s]


# Primal-Dual Tensor Method

In [6]:
eps = 0.02
gamma = eps
image_index = 0

In [7]:
M_p = calculate_lipschitz_constant(n, gamma, p_order=3, A_A_T=A_A_T, device=device)
# M_p = torch.tensor(3.6843e+16, device=device)

In [8]:
# def ot_softmax():
    # log_B = M_over_gamma + torch.outer(psi, torch.ones(n)) + torch.outer(eta, torch.ones(n))
    # max_log_B = log_B.max()
    # log_B_stable = log_B - max_log_B
    # B_stable = torch.exp(log_b_stable)
    
    # return gamma * (torch.log(B_stable.sum()) + max_log_B - lamb @ b)

In [9]:
%%time
optimizer = run_experiment(M_p, gamma, eps, image_index, max_steps=1, device=device)

Step #1, criterion=1.0493743540784806e-08, phi=0.1360974888154517, f=-0.13609749930919524
lambda=tensor([-4.5960e-08, -4.7596e-08, -4.7656e-08,  ..., -4.7656e-08,
        -4.7596e-08, -4.5960e-08], dtype=torch.float64)
x_hat=tensor([1.1084e-03, 3.3733e-05, 1.0267e-06,  ..., 1.0267e-06, 3.3733e-05,
        1.1084e-03], dtype=torch.float64)

CPU times: user 50.3 s, sys: 8.63 s, total: 58.9 s
Wall time: 20.7 s


In [47]:
%%time
optimizer = run_experiment(M_p, gamma, eps, image_index, max_steps=1, device=device)

Step #1, criterion=9.682405933617022e-08, phi=0.006666569841931289, f=-0.006666666665990625
lambda=tensor([-9.0958e-09, -9.0958e-09, -9.0958e-09,  ..., -9.0958e-09,
        -9.0958e-09, -9.0958e-09], dtype=torch.float64)
x_hat=tensor([1.2755e-03, 6.0744e-34, 2.8929e-64,  ..., 2.8929e-64, 6.0744e-34,
        1.2755e-03], dtype=torch.float64)

CPU times: user 56.5 s, sys: 4.15 s, total: 1min
Wall time: 18.9 s


# Fast gradient method for dual function

In [10]:
def fast_gradient_method(
    L, 
    primal_function, 
    dual_function, 
    primal_var_function,
    A_matrix,
    b,
    eps,
    eps_tilde
):
    
    dual_var_next = torch.zeros(2 * n, requires_grad=True, dtype=torch.double)
    y_next = torch.zeros(2 * n, dtype=torch.double)
    z_next = torch.zeros(2 * n, dtype=torch.double)
    k = 0
    one_over_L = 1 / L
    alpha_next = one_over_L
    sum_of_y = y_next
    while True:
        y = y_next.detach().clone()
        z = z_next.detach().clone()
        tau = 2 / (k + 2)
        
        with torch.no_grad():
            dual_var_next = tau * z + (1 - tau ) * y
            dual_var_next.requires_grad_(True)
            # dual_var_next.add_(z, alpha=tau).add_(y, alpha=1 - tau)
        
        phi_next = dual_function(dual_var_next)
        grad_phi_next = torch.autograd.grad(phi_next, dual_var_next)
        y_next = dual_var_next.detach().clone() - one_over_L * grad_phi_next[0]
        
        z_next = z - alpha_next * grad_phi_next[0]
        
        N = k + 1
        
        y_tilde = 1 / (N * (N + 3)) * (sum_of_y + (N + 1) ** 2 * y_next)
        sum_of_y += y_next
        
        primal_var = primal_var_function(dual_var_next)
        F_value = dual_function(y_tilde)
        g_value = primal_function(primal_var)
        cr_1 = abs(F_value + g_value)
        cr_2 = torch.norm(A_matrix @ primal_var - b)
        if k == 0:
            init_cr_1 = cr_1
            init_cr_2 = cr_2
        # print(f'Step #{k}, cr_1={cr_1}, cr_2={cr_2}, phi={F_value}, f={g_value}', end='\n\n')
        clear_output(wait=True)
        print('\n'.join([
            f'Step #{k + 1}', 
            f'cr_1: {init_cr_1} -> {cr_1}',
            f'cr_2: {init_cr_2} -> {cr_2}'
        ]))
        
        if cr_1 <= eps and cr_2 <= eps_tilde:
            break
        
        alpha_next = (k + 2) * one_over_L / 2
        k += 1
        
    return F_value, g_value

In [11]:
images, labels = load_data()

p_list = [34860, 31226, 239, 37372, 17390]
q_list = [45815, 35817, 43981, 54698, 49947]

# x_array = np.linspace(1 / 2e-2, 1 / 4e-4, 6)
# epslist = 1 / x_array
gamma = eps
epsp = eps
p, q = mnist(epsp, p_list[4], q_list[4], images, n)
p = torch.tensor(p)
q = torch.tensor(q)
b = torch.cat((p, q))

In [12]:
M_matrix = calculate_M_matrix(m)

In [13]:
M_1 = calculate_lipschitz_constant(n, gamma, p_order=1, A_A_T=A_A_T, device=device)

In [14]:
half_lamb_len = n
ones = torch.ones(n)
primal_func = lambda x: f(x, M_matrix, gamma, device)
dual_func = lambda lamb: phi(lamb, n, gamma, M_matrix / gamma, ones, p, q)
primal_var_func = lambda lamb: calculate_x(lamb, n, M_matrix / gamma, ones)

In [15]:
eps, gamma

(0.02, 0.02)

In [16]:
fast_gradient_method(M_1, primal_func, dual_func, primal_var_func, A_matrix, b, eps, eps)

Step #31
cr_1: 7.197339946252157e-11 -> 9.199739109400085e-09
cr_2: 0.11877185503486869 -> 0.11877184892671572


KeyboardInterrupt: 

# TEST 

In [45]:
from mnist import MNIST
import numpy as np
from scipy.spatial.distance import cdist
import math
#https://pypi.org/project/python-mnist
mndata = MNIST('./data/')
images, labels = mndata.load_training()
%matplotlib inline
import matplotlib.pyplot as plt
import copy
from numpy import linalg
import time

l = len(images)
n = len(images[0])
m = int(np.sqrt(n))

# def mnist(eps, p, q):
#     p, q = np.float64(images[p]), np.float64(images[q])
#     p, q = p / sum(p), q / sum(q)
    
#     p = (1 - eps / 8) * p + eps / (8 * n)
#     q = (1 - eps / 8) * q + eps / (8 * n)
    
#     return p, q

def cartesian_product(*arrays):
    la = len(arrays)
    dtype = np.result_type(*arrays)
    arr = np.empty([len(a) for a in arrays] + [la], dtype=dtype)
    for i, a in enumerate(np.ix_(*arrays)):
        arr[...,i] = a
    return arr.reshape(-1, la)

C = np.arange(m)
C = cartesian_product(C, C)
C = cdist(C, C)
C /= np.max(C)
C.max()

1.0

In [46]:
one = np.ones(n, np.float64)
I = np.ones([n,n], np.float64)
def f_true(x):
    return (C * x).sum()

def B_round(x):
    r = p_ref / x.dot(one)
    r[r>1] = 1.
    F = np.diag(r).dot(x)
    c = q_ref / (x.T).dot(one)
    c[c>1] = 1.
    F = F.dot(np.diag(c))
    err_r = p_ref - F.dot(one)
    err_c = q_ref - (F.T).dot(one)
    return F + np.outer(err_r, err_c) / abs(err_r).sum()

In [47]:
C.shape

(784, 784)

In [94]:
def phi_(gamma, lamu):
    A = (-C/gamma + np.outer(lamu[:n], one) + np.outer(one, lamu[n:]))
    a = A.max()
    A-=a
    s = a+np.log(np.exp(A).sum())
    return gamma*(-lamu[:n].dot(p) - lamu[n:].dot(q) + s)

def f_(gamma, x):
    y = (x.reshape(-1)).copy()
    y[x.reshape(-1) == 0.] = 1.
    
    y = y.reshape(n, -1)
    return (C * x).sum() + gamma * (x * np.log(y)).sum()

In [124]:
def apd(x0, L):
    betta = 0.
    alpha = 0.
    primal_var = 0.*torch.ones([n, n], dtype=torch.float64)
    z = torch.zeros(2*n, dtype=torch.float64)
    z_new = torch.zeros(2*n, dtype=torch.float64)
    eta = torch.zeros(2*n, dtype=torch.float64)
    eta_new  = torch.zeros(2*n, dtype=torch.float64)
    grad_phi_new = torch.zeros(2*n, dtype=torch.float64)
    one = torch.tensor(one)
    
    # f = lambda x: phi_(gamma, x)
    # f_primal = lambda x: f_(gamma, x)
    f = lambda x: phi(x, n, gamma, M_matrix / gamma, torch.tensor(ones), p, q)
    f_primal = lambda x: f(x, M_matrix, gamma)
    
    k=0
    start_time = time.perf_counter()
    while True:
        # L = L / 2
        # while True:
        alpha_new = (1 + torch.sqrt(4*L*betta + 1)) / 2 / L
        betta_new = betta + alpha_new
        tau = alpha_new / betta_new
        lamu_new = tau * z + (1 - tau) * eta

        logB = (K + torch.outer(lamu_new[:n], one) + torch.outer(one, lamu_new[n:]))
        max_logB =logB.max()
        logB_stable = logB - max_logB

        B_stable = torch.exp(logB_stable)
        u_hat_stable, v_hat_stable = B_stable.dot(one), B_stable.T.dot(one)

        Bs_stable = u_hat_stable.sum()

        phi_new = gamma*(-lamu_new[:n].dot(p) - lamu_new[n:].dot(q) + torch.log(Bs_stable) + max_logB)
        grad_phi_new = gamma*torch.cat((-p + u_hat_stable/Bs_stable, -q + v_hat_stable/Bs_stable),0)


        z_new = z - alpha_new * grad_phi_new
        eta_new = tau * z_new + (1-tau) * eta

        phi_eta = f(eta_new)

            # if phi_eta <= phi_new + grad_phi_new.dot(eta_new - lamu_new) + L * ((eta_new - lamu_new)**2).sum() / 2:
        alpha, betta = alpha_new, betta_new
        z = z_new.copy()
        eta = eta_new.copy()
            #     break    
            # L = L * 2
        primal_var = tau * B_stable/Bs_stable + (1 - tau) * primal_var
        
        cr_1 = abs(f_primal(primal_var) + phi_eta)
        cr_2 = (C * (B_round(primal_var) - primal_var)).sum()
        if k == 0:
            init_cr_1 = cr_1
            init_cr_2 = cr_2

        clear_output(wait=True)
        print('\n'.join([
            f'Step #{k + 1}', 
            f'cr_1: {init_cr_1} -> {cr_1}',
            f'cr_2: {init_cr_2} -> {cr_2}'
        ]))
        if cr_2 <= eps/6 and cr_1 <= eps/6:
            return time.perf_counter() - start_time
        
        k=k+1

In [125]:
x_array = torch.linspace(1 / 2e-2, 1 / 4e-4, 6)
epslist = 1 / x_array

In [126]:
epslist

tensor([0.0200, 0.0019, 0.0010, 0.0007, 0.0005, 0.0004])

In [127]:
# if not os.path.isfile('/content/drive/My Drive/colab/'+'!e_'+'apd_'+str(k)+'.npy'):
# i=0
# for eps in epslist:
k = 0
x0 = torch.zeros(2 * n, dtype=torch.float64)
epsp = epslist[0] / 8
p,q = mnist(epsp, p_list[k], q_list[k], images, n)
p_ref, q_ref = mnist(0, p_list[k], q_list[k], images, n)
gamma=eps/3/torch.log(torch.tensor(n))
K=-C/gamma
y_array[i] = apd(x0, M_1)
# i-=-1
    # np.save('/content/drive/My Drive/colab/'+'!e_'+'apd_'+str(k)+'.npy', y_array)
# print('apd_', k, ' - done', y_array)

UnboundLocalError: local variable 'one' referenced before assignment